In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
var('xx')
Q.<ii> = NumberField(xx^2 + 1)

In [3]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f) * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [4]:
%time adm_SE1 = SE1.find_admissible_projectivities()
len(adm_SE1)

CPU times: user 8.83 s, sys: 243 ms, total: 9.07 s
Wall time: 19.8 s


576

In [5]:
%time simm_SE1 = SE1.find_simmetries(adm_SE1)
len(simm_SE1)

CPU times: user 4.51 s, sys: 181 ms, total: 4.69 s
Wall time: 22.9 s


2

In [6]:
with open('all_L_sets.pickle', 'rb') as fil:
    all_L_sets = pickle.load(fil)

In [7]:
L_set_base = ('E1', 'G4', 'E2', 'G3', 'E3')
possible_L_sets = [L_set for L_set in all_L_sets if L_set[0:3] == ('E1', 'G4', 'E3')]

In [8]:
%time L_set_ideals, all_ideals = SE1.find_conditions_on_L_sets(possible_L_sets, "eck")

CPU times: user 41.7 s, sys: 42.2 ms, total: 41.8 s
Wall time: 41.8 s


In [9]:
L_set_ideals

[[('E1', 'G4', 'E3', 'G6', 'E6'), []],
 [('E1', 'G4', 'E3', 'G6', 'F45'), []],
 [('E1', 'G4', 'E3', 'G6', 'F24'),
  [Ideal (4*c + (-ii)*d + (ii)*e + (-ii + 2)*f, 5*d^2 + (-4*ii - 2)*d*e + (4*ii - 3)*e^2 + (8*ii - 6)*d*f + (36*ii + 38)*e*f + (-24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1,
   Ideal (4*c + (ii)*d + (-ii)*e + (ii + 2)*f, 5*d^2 + (4*ii - 2)*d*e + (-4*ii - 3)*e^2 + (-8*ii - 6)*d*f + (-36*ii + 38)*e*f + (24*ii - 7)*f^2) of Multivariate Polynomial Ring in x, y, z, t, b, c, d, e, f, l, m over Number Field in ii with defining polynomial xx^2 + 1]],
 [('E1', 'G4', 'E3', 'F13', 'E6'), []],
 [('E1', 'G4', 'E3', 'F13', 'E5'), []],
 [('E1', 'G4', 'E3', 'F13', 'E2'), []],
 [('E1', 'G4', 'E3', 'G5', 'F46'), []],
 [('E1', 'G4', 'E3', 'G5', 'E5'), []],
 [('E1', 'G4', 'E3', 'G5', 'F24'),
  [Ideal (4*c + (-ii)*d + (-ii + 2)*e + (ii)*f, 5*d^2 + (8*ii - 6)*d*e + (-24*ii - 7)*e^2 + (-4*ii - 2)*d*f 

In [10]:
ideales = [_ for i in range(len(all_ideals))]
e_sosts = [_ for i in range(len(all_ideals))]
d_sosts = [_ for i in range(len(all_ideals))]
for i in range(len(all_ideals)):
    ideales[i] = all_ideals[i]
    poly_e = ideales[i].gens()[0]
    e_sosts[i] = {e: -(poly_e-poly_e.coefficient(e)*e)/poly_e.coefficient(e)}
    poly_d = [gen for gen in ideales[i].subs(e_sosts[i]).gens() if gen !=0][0]
    d_sosts[i] = {d:-(poly_d-poly_d.coefficient(d)*d)/poly_d.coefficient(d)}
    print([gen.subs(e_sosts[i]).subs(d_sosts[i]) for gen in ideales[i].gens()])

[0, 0]
[0, 0]
[0, 0]
[0, 0]


In [11]:
cubics = [SE1.subs(e_sosts[i]).subs(d_sosts[i]) for i in range(len(ideales))]
main_cubic = cubics[0]
L_sets = [SE1.L_set_base]
for L_set in [get_permuted_L_set(perm) for perm in SE1.find_admissible_permutations()]:
    if L_set not in L_sets:
        L_sets.append(L_set)

In [12]:
%%time
for cubic in cubics:
    L_set, sost = main_cubic.find_projective_equivalence(cubic, L_sets, [c,f])
    print(L_set, sost)
    proj = cubic.find_projectivity(cubic.L_set_base, L_set)[0]
    print(main_cubic == cubic.subs(change_coordinates(proj)).subs(sost))

('E1', 'G4', 'E2', 'G3', 'E3') {c: -c, f: -f}
True
('E1', 'G4', 'E2', 'G6', 'F34') {c: (-ii)*c - f, f: -c + (-ii)*f}
True
('E1', 'G4', 'E2', 'G3', 'E3') {c: (2*ii + 1)*c + (-2*ii - 1)*f, f: -5*c + (4*ii - 3)*f}
True
('E1', 'G4', 'E2', 'G6', 'F34') {c: (2*ii + 1)*c + f, f: -c + (2*ii - 1)*f}
True
CPU times: user 10.3 s, sys: 52.1 ms, total: 10.4 s
Wall time: 10.4 s


In [13]:
SE1_1 = main_cubic

In [14]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 2.87 s, sys: 189 ms, total: 3.06 s
Wall time: 20.9 s


576

In [15]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 1.53 s, sys: 188 ms, total: 1.72 s
Wall time: 7.49 s


4

In [16]:
lines_perms = [Permutation(SE1_1.apply_proj_to_lines(simm[0])).to_permutation_group_element() for simm in simm_SE1_1]
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(4, 'C4')

In [17]:
lines_perms_group.gens_small()

[(2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20)]

In [18]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46')]

In [19]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens() if perm.order()==4]

[('E1', 'G4', 'E3', 'G6', 'F24', 'F46'),
 ('E1', 'G4', 'F34', 'F16', 'E2', 'E6')]

In [20]:
lines_perms_group.gens()

[(),
 (2,3,19,22)(4,21,7,23)(5,26,25,6)(8,16,13,11)(9,12,14,17)(18,24,27,20),
 (2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (2,22,19,3)(4,23,7,21)(5,6,25,26)(8,11,13,16)(9,17,14,12)(18,20,27,24)]